In [10]:
import pandas as pd

손익 = pd.read_csv("/content/drive/MyDrive/리스크 레이더/손익계산서.csv")
손익.drop(['Unnamed: 0', 'sj_nm'], axis=1, inplace=True)
손익.columns

Index(['bsns_year', 'corp_code', 'account_id', 'account_nm', 'thstrm_amount',
       'frmtrm_amount'],
      dtype='object')

In [11]:
손익1 = 손익.loc[손익['thstrm_amount'].notna()]
손익1['account_nm'] = 손익1['account_nm'].str.replace(' ', '', regex=False)
# 손익1 = 손익1[~(손익1['account_id'].str.contains('Other', na=False))]
# 손익1 = 손익1[~(손익1['account_id'] == 'ifrs_EquityAndLiabilities')]
손익1['account_nm'] = 손익1['account_nm'].str.replace(r'\(.*?\)', '', regex=True)

<ipython-input-11-54655456>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  손익1['account_nm'] = 손익1['account_nm'].str.replace(' ', '', regex=False)
<ipython-input-11-54655456>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  손익1['account_nm'] = 손익1['account_nm'].str.replace(r'\(.*?\)', '', regex=True)


In [12]:
account_name_mapping = {}

account_name_mapping = {
    # 매출액 관련 항목
    '매출액': '매출액',
    '총매출액': '매출액',
    'Ⅰ.매출액': '매출액',
    'I.매출액': '매출액',
    '매출': '매출액',
    '매출금': '매출액',

    # 매출원가 관련 항목
    '매출원가': '매출원가',
    'Ⅱ.매출원가': '매출원가',
    'II.매출원가': '매출원가',

    # 매출총이익 관련 항목
    '매출총이익': '매출총이익',
    'Ⅲ.매출총이익': '매출총이익',
    'III.매출총이익': '매출총이익',

    # 판매관리비 관련 항목
    '판매관리비': '판매관리비',
    '판매비와관리비': '판매관리비',
    'Ⅳ.판매비와관리비': '판매관리비',
    'IV.판매비와관리비': '판매관리비',
    '판매비및관리비': '판매관리비',
    '판매및일반관리비': '판매관리비',
    '판매비와일반관리비': '판매관리비',

    # 영업이익 관련 항목
    '영업이익': '영업이익',
    'Ⅴ.영업이익': '영업이익',
    'V.영업이익': '영업이익',
    'Ⅳ.영업이익': '영업이익',
    '영업손익': '영업이익',

    # 당기순이익 관련 항목
    '당기순이익': '당기순이익',
    'Ⅸ.당기순이익': '당기순이익',
    'IX.당기순이익': '당기순이익',
    'Ⅵ.당기순이익': '당기순이익',
    'Ⅰ.당기순이익': '당기순이익',
    '순이익': '당기순이익'
}

손익1['account_nm'] = 손익1['account_nm'].replace(account_name_mapping)

손익1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29831 entries, 0 to 30302
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bsns_year      29831 non-null  int64  
 1   corp_code      29831 non-null  int64  
 2   account_id     29831 non-null  object 
 3   account_nm     29831 non-null  object 
 4   thstrm_amount  29831 non-null  float64
 5   frmtrm_amount  29317 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 1.6+ MB


<ipython-input-12-1978330262>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  손익1['account_nm'] = 손익1['account_nm'].replace(account_name_mapping)


In [13]:
import pandas as pd

# 정확히 일치하는 계정명만 필터링
selected_accounts = [
    '순이익',
    '영업이익',
    '매출액',
    '매출총이익',
    '당기순이익',
    '매출원가',
    '판매관리비'
]

# 필요한 컬럼만 선택하고, 해당 계정명만 필터링
손익_filtered = 손익1[손익1['account_nm'].isin(selected_accounts)].copy()

# 'thstrm_amount'와 'frmtrm_amount'의 콤마 제거 및 숫자형 변환
for col in ['thstrm_amount', 'frmtrm_amount']:
    if pd.api.types.is_numeric_dtype(손익_filtered[col]):
        continue
    손익_filtered[col] = 손익_filtered[col].astype(str).str.replace(',', '', regex=False)
    손익_filtered[col] = pd.to_numeric(손익_filtered[col], errors='coerce')

# 중복 제거: 당기 금액 우선
손익_filtered = 손익_filtered.sort_values(
    by=['bsns_year', 'corp_code', 'account_nm', 'thstrm_amount'],
    ascending=[True, True, True, False]
)
손익_filtered = 손익_filtered.drop_duplicates(subset=['bsns_year', 'corp_code', 'account_nm'], keep='first')

# 당기 피벗
df_thstrm = 손익_filtered.pivot_table(
    index=['corp_code', 'bsns_year'],
    columns='account_nm',
    values='thstrm_amount',
    aggfunc='first'
).reset_index()
df_thstrm.columns = [f'{col}_당기' if col not in ['corp_code', 'bsns_year'] else col for col in df_thstrm.columns]

# 전기 피벗
df_frmtrm = 손익_filtered.pivot_table(
    index=['corp_code', 'bsns_year'],
    columns='account_nm',
    values='frmtrm_amount',
    aggfunc='first'
).reset_index()
df_frmtrm.columns = [f'{col}_전기' if col not in ['corp_code', 'bsns_year'] else col for col in df_frmtrm.columns]

# 병합
손익_pivoted = pd.merge(df_thstrm, df_frmtrm, on=['corp_code', 'bsns_year'], how='outer')

# 결과 확인
print(손익_pivoted.head())
print(손익_pivoted.info())


   corp_code  bsns_year      당기순이익_당기        매출액_당기       매출원가_당기  \
0     101257       2015 -7.349318e+11  5.624113e+11  6.910544e+11   
1     101257       2018  4.221032e+10  2.404580e+11  2.251981e+11   
2     101257       2019  1.852558e+10           NaN  2.515617e+11   
3     101488       2015  1.932074e+10           NaN  3.348295e+11   
4     101488       2016  2.863619e+10           NaN  3.690547e+11   

       매출총이익_당기       영업이익_당기      판매관리비_당기      당기순이익_전기        매출액_전기  \
0 -1.286431e+11 -3.337214e+11  2.050783e+11 -3.549191e+11  9.105774e+11   
1  1.525981e+10  6.284970e+09  8.974838e+09 -2.236206e+11  2.474771e+11   
2  2.475748e+10  1.546670e+10  9.290779e+09  4.221032e+10           NaN   
3  1.134726e+11  2.080051e+10  9.267204e+10  9.082312e+09           NaN   
4  1.370275e+11  3.657418e+10  1.004533e+11  1.932074e+10           NaN   

        매출원가_전기      매출총이익_전기       영업이익_전기      판매관리비_전기  
0  1.135992e+12 -2.254144e+11 -2.615544e+11  3.613998e+10  
1  2.719488e+1

In [14]:
# 결과 CSV 파일로 저장 (선택 사항)
output_pivot_filename = "/content/drive/MyDrive/리스크 레이더/손익계산서_pivot.csv"
손익_pivoted.to_csv(output_pivot_filename, index=False, encoding='utf-8-sig')
print(f"\n✅ 피벗 테이블이 '{output_pivot_filename}' 파일로 저장되었습니다.")



✅ 피벗 테이블이 '/content/drive/MyDrive/리스크 레이더/손익계산서_pivot.csv' 파일로 저장되었습니다.
